In [64]:
from math import sqrt
import re
import numpy as np
import seaborn as sns
import pandas as pd
%matplotlib inline
from pylab import * 
from itertools import product 


In [65]:
ls = range(1,6) #length of kmer
ys = [1, 2, 4, 5, 6, 10, 15] #some threshold
region = "data/region25_out/region25"

### Run profile kernel

In [ ]:
for l, y in product(ls, ys):
    print(">>> Working on parameters L={0}, Y={1}".format(l, y))
    !profkernel-core -o {region}.idfile -K -L {l} -Y {y} -i /usr/share/fastprofkernel/data/Amino.txt -g /usr/share/fastprofkernel/data/Globals.txt {region}.kernelinput > {region}_L{l}_Y{y}.mat 2> {region}_L{l}_Y{y}.mat.log  

>>> Working on parameters L=1, Y=1
>>> Working on parameters L=1, Y=2
>>> Working on parameters L=1, Y=4
>>> Working on parameters L=1, Y=5
>>> Working on parameters L=1, Y=6
>>> Working on parameters L=1, Y=10
>>> Working on parameters L=1, Y=15
>>> Working on parameters L=2, Y=1
>>> Working on parameters L=2, Y=2
>>> Working on parameters L=2, Y=4
^C
>>> Working on parameters L=2, Y=5
>>> Working on parameters L=2, Y=6
>>> Working on parameters L=2, Y=10
>>> Working on parameters L=2, Y=15
>>> Working on parameters L=3, Y=1
>>> Working on parameters L=3, Y=2
>>> Working on parameters L=3, Y=4
>>> Working on parameters L=3, Y=5
>>> Working on parameters L=3, Y=6
>>> Working on parameters L=3, Y=10


### Adding row/col counts to the matrices

In [ ]:
row_count = !wc -l {region}_L{ls[0]}_Y{ys[0]}.mat 
row_count = int(row_count[0].split(" ")[0])
row_count

In [ ]:
for l, y in product(ls, ys):
    !echo "{row_count} {row_count}\n$(cat {region}_L{l}_Y{y}.mat)" > {region}_L{l}_Y{y}.2.mat

### Execute weka

In [ ]:
for l, y in product(ls, ys):
    print(">>> Working on parameters L={0}, Y={1}".format(l, y))
    !java -Xmx5g -cp bin/wekaTobi.jar:bin/weka.jar: weka.classifiers.meta.CVParameterSelection -t {region}.arff -x 5 -S 1 -W weka.classifiers.functions.SMO -- -C 1.0 -L 0.0010 -P 1.0E-12 -N 0 -V -1 -W 1 -K "weka.classifiers.functions.supportVector.CustomPrecomputedKernelMatrixKernelFast -M {region}_L{l}_Y{y}.2.mat" > {region}_L{l}_Y{y}.weka

### extract confusion matrix from WEKA

In [ ]:
perf_dict = {}
for l, y in product(ls, ys):
    file = "{}_L{}_Y{}.weka".format(region, l, y)
    matrix = []
    with open(file) as f: 
        isMatrix = False
        isCV = False
        for line in f.readlines():         
            if line.startswith("=== Stratified cross-validation"):
                isCV = True
            if isCV and line.startswith("=== Confusion Matrix"):
                isMatrix = True
            if isMatrix and isCV: 
                matrix.append(line)
    pos = matrix[3].split("|")[0].strip()
    neg = matrix[4].split("|")[0].strip()
    TP, FN = (int(x.strip()) for x in pos.split())
    FP, TN = (int(x.strip()) for x in neg.split())
    perf_dict[(l, y)] = (TP, FN, FP, FN)

### compare the parameters
#### Which measure to use?
* Accuracy: bad because data is unbalanced
* F-Measure: hard to interpret intuitievely
* MCC: good, easy to interpret and balanced
* Sensistivity, Specificity: good, give the user information about what they actually expect from the predictor

In [ ]:
def mcc(TP, FN, FP, TN): 
    return np.divide(TP * TN + FP * FN, sqrt( (TP+FP) * (TP+FN) * (TN+FP) * (TN+FN)))

In [ ]:
def sens(TP, FN, FP, TN):
    return TP / (TP + FN)

In [ ]:
def spec(TP, FN, FP, TN):
    return TN / (TN + FP)

In [ ]:
def make_df(f):
    f_dict = {}
    for y in ys:
        f_dict[y] = {}
        for l in ls:
            f_dict[y][l] = f(*perf_dict[l, y])
    return pd.DataFrame(f_dict)

In [ ]:
fig, (ax1, ax2, ax3) = subplots(1, 3, figsize=(15, 4))
sns.heatmap(make_df(mcc), ax=ax1)
ax1.set_title("MCC")
sns.heatmap(make_df(sens), ax=ax2)
ax2.set_title("Sens")
sns.heatmap(make_df(spec), ax=ax3)
ax3.set_title("Spec")
for ax in (ax1, ax2, ax3):
    ax.set_xlabel("Y")
    ax.set_ylabel("L")